In [2]:
import pandas as pd
from utils.data_prep_utils import *
from transformers.encoder import *
from transformers.binner import *
from transformers.standardizer import *
from model_binclass.xgboost_binclass import XgboostBinclass

In [ ]:
#Data import form drive
df = pd.read_csv(r'resources/Machine failure.csv')
df = df.drop(['TWF','HDF','PWF', 'OSF', 'RNF', 'UDI', 'Product ID'], axis=1)
target = df.columns[-1]
features = df.columns.drop(target)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Type                     10000 non-null  object 
 1   Air temperature [K]      10000 non-null  float64
 2   Process temperature [K]  10000 non-null  float64
 3   Rotational speed [rpm]   10000 non-null  int64  
 4   Torque [Nm]              10000 non-null  float64
 5   Tool wear [min]          10000 non-null  int64  
 6   Machine failure          10000 non-null  int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 547.0+ KB


In [ ]:
df = data_split_stratified(df, target,  0.2)

encoder_info = learn_encoder(df[df.dataset==1].drop(columns=['dataset', target]))
df = apply_encoder(df, encoder_info, columns_to_drop=['dataset', target])

stand_info = learn_standardizer(df[df.dataset==1].drop(columns=['dataset', target]))
df = apply_standardizer(df, stand_info, columns_to_drop=['dataset', target])

features = df.columns.drop(target, 'dataset')

print(f"y_train - 0: {sum((df.dataset == 1) & (df[target] == 0))}, 1: {sum((df.dataset == 1) & (df[target] == 1))}")
print(f"y_test  - 0: {sum((df.dataset == 0) & (df[target] == 0))}, 1: {sum((df.dataset == 0) & (df[target] == 1))}")

y_train - 0: 7729, 1: 271
y_test  - 0: 1932, 1: 68


In [ ]:
encoder = Encoder()
encoder.learn(df[df.dataset==1], features)
encoder.apply(df)
encoder.save()

In [ ]:
model = XgboostBinclass()
model.learn(df[df.dataset == 1], features, target)
model.apply(df)
model.evaluate(df)
model.explain(df)
model.save()

In [ ]:
model.shap_values

In [6]:
save_encoder(encoder_info, filepath="resources/info/encoder_info.joblib")
save_standardizer(stand_info, filepath="resources/info/stand_info.joblib")
mh.save_model(model_info, filepath="resources/info/model_info.joblib")

Encoder saved to resources/info/encoder_info.joblib
Standardizer saved to resources/info/stand_info.joblib
Model saved to resources/info/model_info.joblib
